In [1]:
import glob
import pandas as pd
import numpy as np
import regex
import pandas_profiling
import pickle

pd.set_option('display.max_columns', 500)
pd.options.display.max_rows = 500

In [2]:
# read in all data to frame using glob

glob_list = glob.glob("data/census/raw/*.csv")

frame = pd.DataFrame()

for file in glob_list:
    df = pd.read_csv(file)
    frame = frame.join(df,  lsuffix='Fact', rsuffix='Fact', how='right')


In [3]:
#remove FactFact but keep one as a header to add back to dataframe
header = frame["FactFact"].iloc[:,0:1]
header1 = np.array(header).tolist()
header_list = [item for sublist in header1 for item in sublist]

#remove columns with Fact or Value in the name
frame1 = frame.filter(regex ='^(?!Fact)')
frame2 = frame1.filter(regex ='^(?!Value)')

#transpose
frame3 = frame2.T

#add header
frame3.columns = [header_list]

#reset index
frame4 = frame3.reset_index()

/anaconda3/lib/python3.7/site-packages/IPython/core/async_helpers.py:67: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


In [4]:
#output to csv
frame4.to_csv("data/census/census_data_prelim.csv")
#then delete weird rows, remove $ sign, remove index, FIPS code

In [6]:
#read in to fix multiindex issue
df = pd.read_csv("data/census/census_data_prelim - census_data_prelim.csv")

#more cleaning
#values too low are set to zero as indicated by letters
df2 = df.replace({'D': 0, 'F':0, 'FN':0, 'NA':0, 'S':0, 'X':0, 'Z':0})

#remove % sign to cast as floats
df3 = df2.replace('%','',regex=True)

In [7]:
# fix city and state names and convert to abbreviation
city = df3['city_state'].str.split(',', expand=True)
city.rename(columns = {0:"closest_city", 1: "closest_state"}, inplace=True)
#manually fix some city names for merge
city['closest_city'] = city['closest_city'].str.replace(' city', '')
city['closest_city'] = city['closest_city'].str.replace(' town', '')
city['closest_city'] = city['closest_city'].str.replace(' CDP', '')
city['closest_city'] = city['closest_city'].str.replace(' county', '')
city['closest_city'] = city['closest_city'].str.replace(' County', '')
city['closest_city'] = city['closest_city'].str.replace(' municipality', '')
city['closest_city'] = city['closest_city'].replace("Baltimore Highlands", 'Baltimore')
city['closest_city'] = city['closest_city'].replace("Indianapolis (balance)", 'Indianapolis')
city['closest_city'] = city['closest_city'].replace("Nashville-Davidson (balance)", 'Nashville')
city['closest_city'] = city['closest_city'].replace("Augusta-Richmond (balance)", 'Augusta')
city['closest_city'] = city['closest_city'].replace("Louisville/Jefferson (balance)", 'Louisville')
city['closest_city'] = city['closest_city'].replace("Athens-Clarke (balance)", 'Louisville')

#manually fix some state names for merge
city['closest_state'] = city['closest_state'].str.replace("Fact", '')
city['closest_state'] = city['closest_state'].str.lstrip(" ")
city['closest_state'] = city['closest_state'].replace("Virginia (County)", 'Virginia')

In [9]:
#convert state name to abbreviation
us_state_abbrev = {
'Alabama': 'AL',
'Alaska': 'AK',
'Arizona': 'AZ',
'Arkansas': 'AR',
'California': 'CA',
'Colorado': 'CO',
'Connecticut': 'CT',
'Delaware': 'DE',
'District of Columbia': 'DC',
'Florida': 'FL',
'Georgia': 'GA',
'Hawaii': 'HI',
'Idaho': 'ID',
'Illinois': 'IL',
'Indiana': 'IN',
'Iowa': 'IA',
'Kansas': 'KS',
'Kentucky': 'KY',
'Louisiana': 'LA',
'Maine': 'ME',
'Maryland': 'MD',
'Massachusetts': 'MA',
'Michigan': 'MI',
'Minnesota': 'MN',
'Mississippi': 'MS',
'Missouri': 'MO',
'Montana': 'MT',
'Nebraska': 'NE',
'Nevada': 'NV',
'New Hampshire': 'NH',
'New Jersey': 'NJ',
'New Mexico': 'NM',
'New York': 'NY',
'North Carolina': 'NC',
'North Dakota': 'ND',
'Northern Mariana Islands':'MP',
'Ohio': 'OH',
'Oklahoma': 'OK',
'Oregon': 'OR',
'Palau': 'PW',
'Pennsylvania': 'PA',
'Puerto Rico': 'PR',
'Rhode Island': 'RI',
'South Carolina': 'SC',
'South Dakota': 'SD',
'Tennessee': 'TN',
'Texas': 'TX',
'Utah': 'UT',
'Vermont': 'VT',
'Virgin Islands': 'VI',
'Virginia': 'VA',
'Washington': 'WA',
'West Virginia': 'WV',
'Wisconsin': 'WI',
'Wyoming': 'WY',
}

#replace state name with state abbreviation
city["closest_state"].replace(us_state_abbrev, inplace=True)

In [27]:
#city['closest_state'].str.encode('utf-8')

In [11]:
#remove city_state because it is not numerical
numbers = df3.drop(["city_state"], axis=1)

#remove commas
numbers2 = numbers.replace(',','',regex=True)

#actual number casting
cols = numbers2.select_dtypes(exclude=['float']).columns
numbers2[cols] = numbers2[cols].apply(pd.to_numeric, downcast='float', errors='coerce')

#set city and state df back to the numbers
df4 = pd.concat([city, numbers2], axis=1, join='inner')

#two exact duplicat
df5 = df4.drop_duplicates()

In [12]:
df5.sort_values("closest_city")

,closest_city,closest_state,"Population estimates, July 1, 2018, (V2018)","Population estimates base, April 1, 2010, (V2018)","Population, percent change - April 1, 2010 (estimates base) to July 1, 2018, (V2018)","Population, Census, April 1, 2010","Persons under 5 years, percent","Persons under 18 years, percent","Persons 65 years and over, percent","Female persons, percent","White alone, percent","Black or African American alone, percent","American Indian and Alaska Native alone, percent","Asian alone, percent","Native Hawaiian and Other Pacific Islander alone, percent","Two or More Races, percent","Hispanic or Latino, percent","White alone, not Hispanic or Latino, percent","Veterans, 2013-2017","Foreign born persons, percent, 2013-2017","Owner-occupied housing unit rate, 2013-2017","Median value of owner-occupied housing units, 2013-2017","Median selected monthly owner costs -with a mortgage, 2013-2017","Median selected monthly owner costs -without a mortgage, 2013-2017","Median gross rent, 2013-2017","Households, 2013-2017","Persons per household, 2013-2017","Living in same house 1 year ago, percent of persons age 1 year+, 2013-2017","Language other than English spoken at home, percent of persons age 5 years+, 2013-2017","Households with a computer, percent, 2013-2017","Households with a broadband Internet subscription, percent, 2013-2017","High school graduate or higher, percent of persons age 25 years+, 2013-2017","Bachelor's degree or higher, percent of persons age 25 years+, 2013-2017","With a disability, under age 65 years, percent, 2013-2017","Persons without health insurance, under age 65 years, percent","In civilian labor force, total, percent of population age 16 years+, 2013-2017","In civilian labor force, female, percent of population age 16 years+, 2013-2017","Total accommodation and food services sales, 2012 (1,000)","Total health care and social assistance receipts/revenue, 2012 (1,000)","Total manufacturers shipments, 2012 (1,000)","Total merchant wholesaler sales, 2012 (1,000)","Total retail sales, 2012 (1,000)","Total retail sales per capita, 2012","Mean travel time to work (minutes), workers age 16 years+, 2013-2017","Median household income (in 2017 dollars), 2013-2017","Per capita income in past 12 months (in 2017 dollars), 2013-2017","Persons in poverty, percent","All firms, 2012","Men-owned firms, 2012","Women-owned firms, 2012","Minority-owned firms, 2012","Nonminority-owned firms, 2012","Veteran-owned firms, 2012","Nonveteran-owned firms, 2012","Population per square mile, 2010","Land area in square miles, 2010"
94,Abilene,TX,122999.0,117512.0,4.700000,117063.0,7.3,23.100000,12.600000,48.299999,73.099998,10.100000,1.1,2.100000,0.0,3.3,26.799999,58.799999,9310.0,6.100000,54.500000,106800.0,1166.0,426.0,846.0,42287.0,2.58,73.300003,19.000000,86.800003,71.699997,85.000000,22.100000,11.500000,17.500000,57.500000,58.700001,0.0,0.0,898344.0,1861865.0,2087352.0,17557.0,15.7,46093.0,22819.0,17.700001,9823.0,5551.0,2991.0,2212.0,7035.0,1262.0,7685.0,1096.199951,106.790001
93,Akron,OH,198006.0,199135.0,-0.600000,199110.0,6.4,21.200001,14.200000,51.900002,60.299999,30.500000,0.3,4.000000,0.0,4.3,2.400000,58.799999,12125.0,5.900000,51.000000,80100.0,981.0,403.0,713.0,84443.0,2.29,85.800003,7.200000,81.199997,70.500000,86.500000,20.799999,11.800000,11.800000,62.299999,59.299999,257415.0,2883921.0,3477813.0,1695828.0,1906638.0,9603.0,21.5,36223.0,22416.0,24.100000,16308.0,8724.0,6308.0,4488.0,11112.0,1546.0,13855.0,3209.699951,62.029999
92,Albuquerque,NM,560218.0,546191.0,2.600000,545852.0,6.3,23.000000,14.200000,51.200001,73.599998,3.300000,4.4,2.700000,0.1,4.3,48.500000,40.000000,38908.0,9.900000,59.799999,188500.0,1324.0,416.0,833.0,221119.0,2.49,83.599998,28.700001,87.199997,76.300003,89.500000,34.299999,9.500000,12.100000,63.799999,59.500000,1578983.0,5183979.0,0.0,4784857.0,9067367.0,16325.0,21.2,49878.0,28229.0,18.200001,42272.0,20208.0,16129.0,16511.0,23116.0,4147.0,35096.0,2907.600098,187.729996
91,Alexa

In [366]:
pandas_profiling.ProfileReport(df5)

Number of variables,57
Number of observations,272
Total Missing (%),0.0%
Total size in memory,66.0 KiB
Average record size in memory,248.3 B
Numeric,39
Categorical,2
Boolean,0
Date,0
Text (Unique),0
Rejected,16


In [13]:
#output to pickle file

pd.to_pickle(df5, 'data/census_data.pkl')